In [1]:
from tqdm._tqdm_notebook import tqdm_notebook
import spacy
from spacy.lang.en import English
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re
import random
from gensim import corpora
import pyLDAvis.gensim
import gensim
import pickle
import numpy as np
import spacy
from tqdm import tqdm
from langdetect import detect
tqdm_notebook.pandas()

In [2]:
spacy.load('en_core_web_sm')
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [4]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\husse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [11]:
df = pd.read_parquet("tweets.parquet")
df = df[['ID','text','topic','datetime']]
df.text = df.text.apply(lambda x:' '.join(re.sub("(\/\w*)|(\\\w*)|(#\S*)"," ",x).split()))
df.text = df.text.apply(lambda x:' '.join(re.sub("(^|\s)((https?:\/\/)?[\w-]+(\.[\w-]+)+\.?(:\d+)?(\/\S*)?)"," ",x).split()))
df.text = df.text.apply(lambda x:' '.join(re.sub("(\w+/+\w*)|(\\+\w*)"," ",x).split()))
df.text = df.text.apply(lambda x:' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))
df.text = df.text.apply(lambda x: re.sub(u'http\S+', u'', x))   
df.text = df.text.apply(lambda x: x.replace(u'RT', u''))
df.drop_duplicates(subset="text",keep='first',inplace=True)

<>:3: DeprecationWarning: invalid escape sequence \/
<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \w
<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \/
<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \w
<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \/
<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \w
<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \S
<ipython-input-11-6e99f0d8b059>:3: DeprecationWarning: invalid escape sequence \/
  df.text = df.text.apply(lambda x:' '.join(re.sub("(\/\w*)|(\\\w*)|(#\S*)"," ",x).split()))
<ipython-input-11-6e99f0d8b059

In [12]:
df.to_parquet("tweets_cleaned.parquet")

In [13]:
df['tokens'] = df['text'].progress_apply(lambda x:prepare_text_for_lda(x))

In [14]:
df.to_parquet("tweets_tokenized.parquet")

In [369]:
df = pd.read_parquet("tweets_tokenized.parquet")

In [370]:
df[df["topic"]=="3"]

,ID,text,topic,datetime,tokens
0,1197756333438722049,Orion Rising,3,2019-11-22 07:58:57,"[orion, rising]"
3,1197756367009800199,Gusto mong malaman saan napunta ang pera ng go...,3,2019-11-22 07:59:05,"[gusto, malaman, napunta, gobyerno, bilbil, do..."
7,1197756378871410689,e em quanto tempo ela deve falar que o clipe d...,3,2019-11-22 07:59:08,"[quanto, tempo, falar, clipe]"
8,1197756379953422336,Dalawang kembot na lang nasa taytay na ko,3,2019-11-22 07:59:08,"[dalawang, kembot, taytay]"
13,1197756397338812417,Nakaka kaba pala talaga kapag 18 ka na tapos n...,3,2019-11-22 07:59:13,"[nakaka, talaga, kapag, tapos]"
18,1197756409217089536,punta ba kamu dyan nasa manila lang kami oh ha...,3,2019-11-22 07:59:15,"[punta, manila, hahaha]"
22,1197756416141869057,12000674 NASA 11 12 Journal of Geoph,3,2019-11-22 07:59:17,"[12000674, journal, geoph]"
25,1197756419551977472,Cient fico de la NASA afirma haber detectado v...,3,2019-11-22 07:59:18,"[cient, afirma, haber, detectado, marte, marte]"
27,1197756426426314752,halo halo na yung nasa utak namin kanina nung ...,3,2019-11-22 07:59:19,"[namin, kanina, accounting]"
38,1197756451973926913,The Orion Rising via Published November 22 201...,3,2019-11-22 07:59:26,"[orion, rising, publish, november]"


In [371]:
text_data = df[df["topic"]=="3"]['tokens'].tolist()

In [372]:
dictionary = corpora.Dictionary(text_data)

In [373]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [374]:
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [375]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10n.gensim')

In [376]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.026*"header" + 0.021*"crush" + 0.019*"game" + 0.018*"nalang" + 0.018*"thank"')
(1, '0.048*"hahaha" + 0.029*"tamang" + 0.021*"tatu" + 0.020*"twitter" + 0.013*"video"')
(2, '0.017*"taong" + 0.016*"mahal" + 0.013*"paligid" + 0.012*"maging" + 0.010*"lag"')
(3, '0.038*"likod" + 0.026*"bukas" + 0.019*"sorry" + 0.017*"tweet" + 0.016*"gitna"')
(4, '0.013*"today" + 0.011*"space" + 0.010*"office" + 0.009*"center" + 0.009*"baguio"')
(5, '0.021*"space" + 0.015*"earth" + 0.014*"right" + 0.012*"november" + 0.011*"second"')
(6, '0.022*"super" + 0.020*"hahahaha" + 0.012*"guppy" + 0.012*"company" + 0.011*"orion"')
(7, '0.039*"hindi" + 0.029*"naman" + 0.023*"lahat" + 0.021*"kahit" + 0.020*"talaga"')
(8, '0.020*"status" + 0.018*"freeze" + 0.012*"happy" + 0.011*"kwarto" + 0.011*"needy"')
(9, '0.045*"naman" + 0.030*"tapos" + 0.028*"talaga" + 0.026*"bahay" + 0.023*"kanina"')


In [377]:
words = set()
for topic in topics:
    words.update(re.findall('"([^"]*)"', topic[1]))

In [378]:
words

{'baguio',
 'bahay',
 'bukas',
 'center',
 'company',
 'crush',
 'earth',
 'freeze',
 'game',
 'gitna',
 'guppy',
 'hahaha',
 'hahahaha',
 'happy',
 'header',
 'hindi',
 'kahit',
 'kanina',
 'kwarto',
 'lag',
 'lahat',
 'likod',
 'maging',
 'mahal',
 'nalang',
 'naman',
 'needy',
 'november',
 'office',
 'orion',
 'paligid',
 'right',
 'second',
 'sorry',
 'space',
 'status',
 'super',
 'talaga',
 'tamang',
 'taong',
 'tapos',
 'tatu',
 'thank',
 'today',
 'tweet',
 'twitter',
 'video'}

In [379]:
np.save("words_b",words)

In [380]:
df["inf"] = 0

In [381]:
df["inf"] = df[df["topic"]=="3"]["text"].apply(lambda x:jaccard(tokenize(x),list(words)))

In [382]:
df = df.sort_values(by="inf",ascending=False)

In [383]:
df = df.reset_index()

In [389]:
news = df['text'][df['lang'].str.contains('en',na=False)].iloc[:20].tolist()

In [390]:
news

['NASA s Super Guppy aircraft on final approach into Mansfield Airport today Orion',
 'NASA s Super Guppy just landed today in Mansfield Ohio carrying the Orion capsule http',
 'AYAN NALANG AVAIL NUNG AKO NA NASA PILA JUSKO HINDI NA LAHAT AVAILABLE YUNG NASA GITNA NA VIP SEATED',
 'HAHASHSHSHSJSHAHAHAHA AMPOTEK TALAGA NI ALEX KANINA NASA TUKTOK NA KAME NG SPACE SHUTTLE AMPS TAPOS SYA NA LANG NAGSASASALITA ANG TAGAL NAMAN NETO HAHAHAJAHAHSHAHAHSHSHWHWBBWHAHAHAHA ANG BAGAL BAGAL NAMAN KASI PREPARED NA KAMI BUMULUSOK E PINAPATAGAL PA',
 ' 9 super guppy lands mansfield carrying orion space capsule Super Guppy lands in Mansfield carrying NASA s new space capsule',
 'NASA Super Guppy space capsule set to land in Mansfield on Sunday  9 guppy bring orion spacecraft lahm airport sunday',
 'Super Guppy lands in Mansfield carrying NASA s new space capsule  9 super guppy lands mansfield carrying orion space capsule',
 'NASA Super Guppy lands in Mansfield carrying NASA s new space capsule  9 super 

In [230]:
for line in news:
    print("-"+"".join(re.sub(' s '," ",line)))

-Tesla Cybertruck Elon Musk hints at making smaller version of the electric pickup truck  la cybertruck elon musk hints smaller electric pickup truck utm source dlvr it utm medium twitter
-Elon Musk explains why Tesla windows smashed during presentation  81489 cybertruck glass window fail elon musk explanation utm campaign theverge utm content entry utm medium social utm source twitter via
-Elon Musk explains why Tesla Cybertruck windows smashed during presentation  81489 cybertruck glass window fail elon musk explanation utm campaign theverge utm content entry utm medium social utm source twitter via
-Elon Musk revealed Tesla long promised electric pickup truck dubbed the Cybertruck this week in Los Angeles near Space X rocket factory  debuts f utm source twitter utm medium social utm campaign organic
-The new electric Cybertruck designed by the Elon Musk company Tesla Slight problem with the unbreakable glass though Would you buy one or what
-I fucking love it Elon Musk just announce

In [32]:
str1 = df[df["ID"]=="1198891003484213248"].text.tolist()

In [54]:
str2 = df[df["ID"]=="1198949887800422401"].text.tolist()

In [55]:
len(np.intersect1d(tokenize(str1[0]),tokenize(str2[0])))

32

In [56]:
len(tokenize(str2[0]))

48

In [30]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [66]:
jaccard(tokenize(str1[0]),tokenize(str2[0]))

0.5161290322580645

In [340]:
df["show"] = True

In [341]:
count = 1
for i,r in df[df['topic']=="4"].iterrows():
    for j in range(i,0,-1):
        if (df.iloc[i].show) and (df.iloc[i-j].show) and jaccard(tokenize(df.iloc[i].text),tokenize(df.iloc[i-j].text))>0.2:
            df["show"].iloc[i]=False
            break
    if (df.iloc[i].show):
        count = count + 1
    if (count>10):
        break

In [349]:
df[df['show']]['text'].tolist()

['UK ELECTION BREXIT The leader of Britain s opposition Labour Party Jeremy Corbyn dismisses criticism of his decision to remain neutral if he were the prime minister and a second Brexit referendum were held STORY NUMBER 4241402',
 'World Boris Johnson unveils manifesto for Brexit Britain LONDON British Prime Minister Boris Johnson unveils his Conservative Party s manifesto on Sunday pledging to move on from Brexit and austerity in a bid to secure a general election majori ',
 'Sorry people are still going to vote Brexit Party in the North instead of Labour Neutral Jeremy Corbyn s campaign is bombing claims Nigel Farage  7 717875 ito amp    ',
 'BREXIT LIVE No going back now EU sets up first European Commission WITHOUT Britain  1208864 latest news tory manifesto boris johnson nigel farage brexit party election news',
 'No deal Brexit planning will continue if Boris Johnson wins election Tory minister admits  cs   Social   ',
 'Boris Johnson to unveil Tory manifesto to get Brexit done  

In [153]:
detect(df.iloc[11].text)

'fr'

In [152]:
df.iloc[11].text

'Klopp Le meilleur joueur de l histoire c est Messi mais le meilleur joueur de la saison derni re c est Virgil van Dijk il devrait remporter le Ballon d Or Je ne sais pas exactement comment a fonctionne mais c est comme a que je le vois'

In [155]:
if (detect(df.iloc[11].text)=='en'):
    print("L")

In [387]:
df["lang"] = df["text"].progress_apply(lambda x:det(x) if x else None)

In [386]:
def det(x):
    try:
        return detect(x)
    except:
        return ""

In [348]:
df['text'] = df['text'].str.replace("http","").str.replace("utm","").str.replace("source",'').str.replace("twitter","").str.replace("utm","").str.replace("medium","").str.replace("siteshare","").str.replace('social','').str.replace('via','').str.replace("Twitter",'').str.replace('html','').str.replace('share','').str.replace('top','')

In [104]:
df.to_parquet("final.parquet")

In [350]:
df['lang']='en'

In [351]:
df[:100].to_parquet("brexit.parquet")

In [121]:
df[df['datetime']>'2019-11-25']

,index,ID,text,topic,datetime,tokens,inf,show,lang
0,341406,1198854697244864512,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 08:43:28,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.114286,True,en
1,319689,1198740177293860871,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 01:08:24,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.112676,False,en
2,315652,1198726961574678529,VIDEO Jurgen Klopp believes Virgil van Dijk sh...,0,2019-11-25 00:15:53,"[video, jurgen, klopp, belief, virgil, ballon,...",0.112500,False,en
3,360868,1198923524888502272,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 13:16:57,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.111111,False,en
4,350363,1198891003484213248,Liverpool manager J rgen Klopp has claimed tha...,0,2019-11-25 11:07:44,"[liverpool, manager, klopp, claim, virgil, pla...",0.107527,False,en
5,370918,1198949887800422401,Liverpool manager J rgen Klopp has claimed tha...,0,2019-11-25 15:01:43,"[liverpool, manager, klopp, claim, virgil, pla...",0.105263,False,en
6,372427,1198953785726504960,Jurgen Klopp believes deserves to win Ballon d...,0,2019-11-25 15:17:12,"[jurgen, klopp, belief, deserve, ballon, ahead...",0.104478,False,en
7,387282,1198990978977153025,Who should win the Ballon d Or Lionel Messi Cr...,0,2019-11-25 17:45:00,"[ballon, lionel, messi, cristiano, ronaldo, vi...",0.100000,False,en
8,356944,1198911726848299008,If you give it to the best player of last seas...,0,2019-11-25 12:30:05,"[player, season, virgil, jurgen, klopp, back, ...",0.098765,False,en
9,378263,1198968355195097088,Kylian Mbappe backs Lionel Messi to claim reco...,0,2019-11-25 16:15:06,"[kylian, mbappe, back, lionel, messi, claim, r...",0.098592,False,en


In [136]:
df.to_parquet("df.parquet")

In [156]:
df = pd.read_parquet('df.parquet')

In [157]:
df

,index,ID,text,topic,datetime,tokens,inf,show,lang
0,341406,1198854697244864512,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 08:43:28,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.114286,True,en
1,319689,1198740177293860871,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 01:08:24,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.112676,False,en
2,315652,1198726961574678529,VIDEO Jurgen Klopp believes Virgil van Dijk sh...,0,2019-11-25 00:15:53,"[video, jurgen, klopp, belief, virgil, ballon,...",0.112500,False,en
3,360868,1198923524888502272,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 13:16:57,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.111111,False,en
4,350363,1198891003484213248,Liverpool manager J rgen Klopp has claimed tha...,0,2019-11-25 11:07:44,"[liverpool, manager, klopp, claim, virgil, pla...",0.107527,False,en
5,370918,1198949887800422401,Liverpool manager J rgen Klopp has claimed tha...,0,2019-11-25 15:01:43,"[liverpool, manager, klopp, claim, virgil, pla...",0.105263,False,en
6,372427,1198953785726504960,Jurgen Klopp believes deserves to win Ballon d...,0,2019-11-25 15:17:12,"[jurgen, klopp, belief, deserve, ballon, ahead...",0.104478,False,en
7,387282,1198990978977153025,Who should win the Ballon d Or Lionel Messi Cr...,0,2019-11-25 17:45:00,"[ballon, lionel, messi, cristiano, ronaldo, vi...",0.100000,False,en
8,356944,1198911726848299008,If you give it to the best player of last seas...,0,2019-11-25 12:30:05,"[player, season, virgil, jurgen, klopp, back, ...",0.098765,False,en
9,378263,1198968355195097088,Kylian Mbappe backs Lionel Messi to claim reco...,0,2019-11-25 16:15:06,"[kylian, mbappe, back, lionel, messi, claim, r...",0.098592,False,en


In [158]:
final = pd.read_parquet("final.parquet")

In [159]:
final

,index,ID,text,topic,datetime,tokens,inf,show,lang
0,341406,1198854697244864512,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 08:43:28,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.114286,True,en
1,319689,1198740177293860871,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 01:08:24,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.112676,False,en
2,315652,1198726961574678529,VIDEO Jurgen Klopp believes Virgil van Dijk sh...,0,2019-11-25 00:15:53,"[video, jurgen, klopp, belief, virgil, ballon,...",0.112500,False,en
3,360868,1198923524888502272,Liverpool boss Jurgen Klopp believes Virgil va...,0,2019-11-25 13:16:57,"[liverpool, jurgen, klopp, belief, virgil, bal...",0.111111,False,en
4,350363,1198891003484213248,Liverpool manager J rgen Klopp has claimed tha...,0,2019-11-25 11:07:44,"[liverpool, manager, klopp, claim, virgil, pla...",0.107527,False,en
5,370918,1198949887800422401,Liverpool manager J rgen Klopp has claimed tha...,0,2019-11-25 15:01:43,"[liverpool, manager, klopp, claim, virgil, pla...",0.105263,False,en
6,372427,1198953785726504960,Jurgen Klopp believes deserves to win Ballon d...,0,2019-11-25 15:17:12,"[jurgen, klopp, belief, deserve, ballon, ahead...",0.104478,False,en
7,387282,1198990978977153025,Who should win the Ballon d Or Lionel Messi Cr...,0,2019-11-25 17:45:00,"[ballon, lionel, messi, cristiano, ronaldo, vi...",0.100000,False,en
8,356944,1198911726848299008,If you give it to the best player of last seas...,0,2019-11-25 12:30:05,"[player, season, virgil, jurgen, klopp, back, ...",0.098765,False,en
9,378263,1198968355195097088,Kylian Mbappe backs Lionel Messi to claim reco...,0,2019-11-25 16:15:06,"[kylian, mbappe, back, lionel, messi, claim, r...",0.098592,False,en


In [307]:
df['show'].iloc[38]=False

In [306]:
df[df['show']]

,index,ID,text,topic,datetime,tokens,inf,show,lang
0,118052,1198045135931674624,The Trump administration is withholding over 1...,2,2019-11-23 03:06:33,"[trump, administration, withholding, million, ...",0.128205,True,en
37,243432,1198512348761337856,The Ride of Freedom Lebanese Independence Day ...,2,2019-11-24 10:03:05,"[freedom, lebanese, independence, parade, leba...",0.095238,True,en
38,387124,1198990538881548289,MajalatForum19 is gathering civil associations...,2,2019-11-25 17:43:15,"[majalatforum19, gathering, civil, association...",0.094595,True,en
43,385189,1198985627351556097,Exactly security assistance to Lebanon is inte...,2,2019-11-25 17:23:44,"[exactly, security, assistance, lebanon, inten...",0.093023,True,en
49,339804,1198845156469944320,Lebanese security forces fire tear gas amid co...,2,2019-11-25 08:05:33,"[lebanese, security, force, confrontation, cen...",0.092105,True,en
70,262183,1198572430316494849,Examples please What did NATO USA and Britain ...,2,2019-11-24 14:01:50,"[example, please, britain, syria, egypt, leban...",0.086022,True,en
72,135144,1198134815133822977,Trump also holds up military assistance to Leb...,2,2019-11-23 09:02:54,"[trump, hold, military, assistance, lebanon, i...",0.085714,True,en
74,372366,1198953639311740929,I can t believe there are no why did congress ...,2,2019-11-25 15:16:37,"[believe, congress, approve, million, military...",0.085714,True,en
75,95936,1197983373047799810,Trump caught DELAYING AID to yet another count...,2,2019-11-22 23:01:08,"[trump, catch, delay, another, country, despit...",0.085106,True,en
76,354848,1198905435027202048,Is this Ukraine Trump s OMB holds up congress ...,2,2019-11-25 12:05:04,"[ukraine, trump, hold, congress, approve, mili...",0.084906,True,en
